<a href="https://colab.research.google.com/github/kenneth-craig/academyPub/blob/main/ML-XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#upload sonar.csv file from local drive and import file
# load and summarize the sonar dataset
from google.colab import files
import pandas as pd
import numpy as np

diabities_data = files.upload()

Saving iris.csv to iris.csv


In [6]:
!pwd
!ls

/content
iris.csv  sample_data


# Simple XGBoost Problem
**Using pima indians diabetes dataset**

In [3]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

ds = np.loadtxt('/content/pima-indians-diabetes.data.csv', delimiter=',')

# Split into X & Y datset
# X is columns 0..7
# & is column 8 transposed
X = ds[:,0:8]
Y = ds[:,8]

# Split into TRAIN & TEST datasets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=7)

#fit model on training data
model = XGBClassifier()
model.fit(X_train, Y_train)

# make predictions for the test data
predictions = model.predict(X_test)

# evaluation predictions
accuracy = accuracy_score(Y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100))

OSError: ignored

# Using XGBoost to Predict Iris flower

In [50]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

#load dataset
df = pd.read_csv('iris.csv', header=None)
ds = df.values

# Split the data into X and Y
X = ds[:,0:4]
Y = ds[:,4]

# Encode string class values as integers
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(Y)
le_y = label_encoder.transform(Y)

X_train, X_test, y_train, y_test = train_test_split(X, le_y, test_size=0.33, random_state=7)

# Fit model on training data
model = XGBClassifier()
model.fit(X_train, y_train)

#make predictions for test data
predictions = model.predict(X_test)

# Evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100))



Accuracy: 92.00%
